In [2]:
!pip install -q lightgbm

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb

In [4]:
# Cargar dataset
# Utilizamos un dataset incluido en seaborn como ejemplo: 'mpg' (consumo de gasolina)
df = sns.load_dataset('mpg').dropna()
df = df.select_dtypes(include=[np.number])  # solo variables numéricas

In [ ]:
df.head()

In [5]:
# Definir variables
X = df.drop('mpg', axis=1)
y = df['mpg']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Entrenar modelos

## Regresión lineal
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

## Árbol de decisión para regresión
tree = DecisionTreeRegressor(random_state=42, max_depth=5)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

## LightGBM
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt'
}

gbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=[lgb_eval])
y_pred_lgb = gbm.predict(X_test)

In [ ]:
# Evaluar resultados

def eval_model(y_test, y_pred, name):
    print(f'🔹 {name}')
    print(f'  RMSE: {mean_squared_error(y_test, y_pred):.2f}')
    print(f'  R²:   {r2_score(y_test, y_pred):.2f}\n')

eval_model(y_test, y_pred_lr, 'Regresión Lineal')
eval_model(y_test, y_pred_tree, 'Árbol de Decisión')
eval_model(y_test, y_pred_lgb, 'LightGBM')

In [ ]:
# Visualización
plt.figure(figsize=(10, 5))
plt.plot(y_test.values, label='Real', marker='o')
plt.plot(y_pred_lr, label='Linear', marker='.')
plt.plot(y_pred_tree, label='Tree', marker='x')
plt.plot(y_pred_lgb, label='LightGBM', marker='s')
plt.legend()
plt.title("Predicciones de modelos de regresión")
plt.grid()
plt.show()